### 安裝環境

In [ ]:
! pip install langchain huggingface_hub transformers sentencepiece sentence_transformers accelerate unstructured chromadb

Transformers 庫中最基本的對象是`pipeline()`函數。它將模型與其必要的預處理和後處理步驟連接起來，使我們能夠通過直接輸入任何文字並獲得最終的答案：

[models](https://huggingface.co/models)

In [ ]:
##情感分析
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("今天天氣很好")
# classifier(
#     ["This world is very dirty","This food is very delicious"]
# )

In [ ]:
##文字生成
from transformers import pipeline

generator = pipeline("text-generation") #你也可以使用自己想用的模型https://huggingface.co/models
generator("In this course, we will teach you how to")

### 範例一 Taiwan-LLM-7B模型
https://huggingface.co/yentinglin/Taiwan-LLM-7B-v2.0.1-chat

In [ ]:
# pip install transformers>=4.34
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="yentinglin/Taiwan-LLM-7B-v2.0.1-chat", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "你是一個人工智慧助理",
    },
    {"role": "user", "content": "東北季風如何影響台灣氣候？"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

### 範例二 聯發科Breeze-7B模型
https://huggingface.co/MediaTek-Research/Breeze-7B-Instruct-v1_0

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Instruction Model
model = AutoModelForCausalLM.from_pretrained(
    "MediaTek-Research/Breeze-7B-Instruct-v1_0",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2" # optional
)

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/508M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1365: UserWarning: Current model requires 50332032 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("MediaTek-Research/Breeze-7B-Instruct-v1_0")
chat = [
  {"role": "user", "content": "你好，請問你可以完成什麼任務?" },
  {"role": "assistant", "content": "你好，我可以幫助您解決各種問題、提供資訊和協助您完成許多不同的任務。例如：回答技術問題、提供建議、翻譯文字、尋找資料或協助您安排行程等。請告訴我如何能幫助您。"},
  {"role": "user", "content": "東北季風如何影響台灣氣候?"},
]
tokenizer.apply_chat_template(chat, tokenize=False)
outputs = model.generate(tokenizer.apply_chat_template(chat, return_tensors="pt"),
                         # adjust below parameters if necessary
                         max_new_tokens=128,
                         top_p=0.01,
                         top_k=85,
                         repetition_penalty=1.1,
                         temperature=0.01)

print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `85` -- this flag is onl

<s>  You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.  [INST] 你好，請問你可以完成什麼任務? [/INST] 你好，我可以幫助您解決各種問題、提供資訊和協助您完成許多不同的任務。例如：回答技術問題、提供建議、翻譯文字、尋找資料或協助您安排行程等。請告訴我如何能幫助您。 [INST] 東北季風如何影響台灣氣候? [/INST] 東北季風是一種冬季的主要風系，主要由西伯利亞冷空氣所形成，在東亞地區具有重要的氣候影響力。它對臺灣的氣候有哪些影響呢？

1. 氣溫下降：東北季風帶來的冷空氣會使臺灣的氣溫降低，尤其是在冬天，北部和東部地區感受最明顯。這使得這些地區的冬季較寒冷，而南部和西部地區則受到地形屏障，降溫程度較小。
2. 降雨增加：東北季風常伴隨著水氣，因此當它吹到臺灣時，容易為島內帶來較多的降雨


### 範例三 Meta-Llama-3
https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a friendly chatbot. The main customer group you are dealing with is Taiwanese. Please reply to them in Traditional Chinese."},
    {"role": "user", "content": "東北季風如何影響台灣氣候?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

# 檢索增強生成流程範例

### 構建RAG檢索

In [ ]:
# import
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# 載入文件
loader = DirectoryLoader("/content/", glob="**/*.txt")
documents = loader.load()

# 將其分割成片段
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 創建開源的embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 把資料匯入Chromadb
db = Chroma.from_documents(docs, embedding_function)

### 選擇模型

In [ ]:
##Breeze-7B
import torch
from transformers import pipeline, AutoTokenizer
from langchain.llms import HuggingFacePipeline

model = "MediaTek-Research/Breeze-7B-Instruct-v1_0"
tokenizer = AutoTokenizer.from_pretrained("MediaTek-Research/Breeze-7B-Instruct-v1_0") #, use_fast=True)
pipe = pipeline("text-generation",
          model=model,
          batch_size=512,
          torch_dtype=torch.bfloat16,
          device_map='auto',
          max_new_tokens=256,
          temperature=0.5,
          do_sample=True,
          top_k=85,
          top_p=0.01,
          repetition_penalty=1.1)

llm = HuggingFacePipeline(pipeline=pipe)

### 生成回答

In [ ]:
#生成問答
from langchain import VectorDBQA
qa = VectorDBQA.from_chain_type(llm=llm, chain_type= "stuff", vectorstore=db, return_source_documents=True)
result = qa({"query": "2024總統候選人有誰?"})
print(result['result'])